In [2]:
import numpy as np
import pandas as pd
import scipy.cluster.hierarchy as shc
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
import seaborn as sb

In [38]:
df5 = pd.read_csv('../../WFc2.vcf', sep = '\t')
df4 = pd.read_table('../Sequence and SNP data/Bout340can.tsv', header = None)
df4['Start'] = df4[[8,9]].min(axis = 1)
df4['End'] = df4[[8,9]].max(axis = 1)
t = []
for s,e in df4[[8,9]].itertuples(index = False):
    if s < e:
        t.append('forward')
    else:
        t.append('reverse')
df4['Strand'] = t

t = []
for c, p in df5[['#CHROM', 'POS']].itertuples(index = False):
    f = False
    for l,o,s,e in df4[df4[1] == c][[0, 'Strand', 'Start', 'End']].itertuples(index = False):
        if s <= p <= e:
            t.append((l, o, s, e))
            f = True
            break
    if f == False:
        t.append(('None', o, s, e))
        
df5['Gene'] = list(zip(*t))[0]
df5['Strand'] = list(zip(*t))[1]
df5['Start'] = list(zip(*t))[2]
df5['End'] = list(zip(*t))[3]

df5 = df5[df5['Gene'] != 'None']

t = []
for i,r in df5.iterrows():
    if ',' in r['ALT']:
        for a in r['ALT'].split(','):
            r2 = r.copy()
            r2.loc['ALT'] = a
            t.append(r2)
    else:
        t.append(r)
        
df6 = pd.DataFrame(t)
df6 = df6.applymap(lambda x: 0 if x == '0/0' else x).applymap(lambda x: str(x).count('/0') if '/0' in str(x) else x).applymap(lambda x: str(x).count('0/') if '0/' in str(x) else x).applymap(lambda x: 2 if '/' in str(x) else x)

In [41]:
df6.drop(columns = ['Gene', 'Strand', 'Start', 'End'])#.to_csv('../../WFC2L.vcf', sep = '\t', index = False)

In [48]:
df3 = pd.read_csv('../Figures and tables/340 substitution effects annotated.csv')

In [66]:
df4 = pd.merge(df, df3[['ID', 'MID', 'ALT']], on = ['ID', 'ALT']).drop_duplicates()

In [68]:
df4['ID'] = df4['MID']

In [70]:
df4.drop(columns = ['MID']).rename(columns = {'ID' : 'MID'}).to_csv('../../WFC2L.vcf', sep = '\t', index = False)

In [86]:
df = pd.read_csv('../../WFC2L.vcf', sep = '\t')

In [87]:
df2 = pd.read_csv('../Figures and tables/t-test.csv')

In [88]:
df2['FDR adjusted p value'] = multipletests(df2['p value'], alpha=0.01, method='fdr_bh', returnsorted=True)[1]

In [89]:
df2 = df2[df2['FDR adjusted p value'] < 0.01]

In [91]:
df2[df2['Effect'] == 'missense'].sort_values('p value').drop_duplicates('QTL')

,MID,AA,aa,t statistic,p value,Trait,Frameshift,Effect,Site,Transcript,ID,QTL,Expression cluster,Functionally annotated,Outlier in expression,annotation,FDR adjusted p value
1,7539,120,27,7.772573,1.297483e-12,Width_WO_husk,no,missense,exon,LOC_Os05g08900.1,5698868,qCDP5,2,No,No,expressed protein,4.166594e-07
29,10414,56,104,7.056276,5.073727e-11,Filtered_Shoot_length,no,missense,exon,LOC_Os06g35560.1,7536282,qCDP7,1,No,No,"reticuline oxidase-like protein precursor, put...",7.735442e-07
125,15503,137,35,6.735743,2.413577e-10,Width_WO_husk,no,missense,exon,LOC_Os08g43370.1,10404140,qCDP9,9,Yes,No,"6-phosphogluconolactonase, putative, expressed",1.211046e-06
472,26542,138,32,5.693634,5.435548e-08,Stress_Shoot_weight,no,missense,exon,LOC_Os12g41620.1,15176951,qCDP17,1,No,No,"WD domain, G-beta repeat domain containing pro...",7.334085e-05
564,678,140,18,-5.555958,1.165810e-07,Filtered_Stress_Shoot_length,no,missense,exon,LOC_Os01g66150.1,1369440,qCDP1,2,No,No,expressed protein,1.290632e-04
680,17597,124,43,-5.486069,1.520233e-07,Lost_Root_length,no,missense,exon,LOC_Os09g19160.1,11026235,qCDP11,2,No,No,"serine/threonine-protein kinase, putative, exp...",1.417246e-04
768,3768,23,134,-5.447102,1.970189e-07,Filtered_Length_WO_husk,no,missense,exon,LOC_Os03g03390.1,2844635,qCDP3,2,No,No,"G-patch domain containing protein, expressed",1.645474e-04
941,23399,86,71,-5.294792,4.014997e-07,Filtered_Length_WO_husk,no,missense,exon,LOC_Os11g26830.1,13142183,qCDP14,7,No,No,"ATP-binding region, ATPase-like domain contain...",2.737435e-04
1142,12481,161,12,5.129400,7.817583e-07,Width_WO_husk,no,missense,exon,LOC_Os07g37210.1,8918219,qCDP8,1,Yes,No,"MYB family transcription factor, putative, exp...",4.388903e-04
1174,16907,132,37,5.103247,9.004027e-07,Shoot_weight,no,missense,exon,LOC_Os09g15570.1,10905177,qCDP10,2,No,No,"OsFBX315 - F-box domain containing protein, ex...",4.921624e-04


In [75]:
df.index = df['MID'].values

In [77]:
df3 = df.iloc[:, 9:]

In [78]:
df3

,IRIS_313-10340,IRIS_313-10341,IRIS_313-10537,IRIS_313-10539,IRIS_313-10587,IRIS_313-10592,IRIS_313-10593,IRIS_313-10595,IRIS_313-10598,IRIS_313-10600,...,IRIS_313-9397,IRIS_313-9422,IRIS_313-9445,IRIS_313-9594,IRIS_313-9606,IRIS_313-9617,IRIS_313-9626,IRIS_313-9661,IRIS_313-9682,IRIS_313-9696
0,2,2,0,2,2,2,2,2,2,2,...,2,2,0,2,2,2,2,2,0,2
1,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2,0,2,2,2,2,2,2,2,...,2,2,0,2,2,2,2,2,0,2
3,2,2,0,2,2,2,2,2,2,2,...,2,2,0,2,2,2,2,2,0,2
4,2,2,0,2,2,2,2,2,2,2,...,2,2,0,2,2,2,2,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28183,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28184,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,1,2,2,2
28185,2,2,0,0,0,0,0,0,2,0,...,2,2,0,0,0,2,0,2,2,0
28186,0,0,0,2,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0


In [109]:
tl = df2[(df2['Effect'] == 'synonymous') & (df2['QTL'] == df3[df3['ID'] == eyed]['QTL'].to_list()[0])].drop_duplicates('ID')['ID'].astype(str).to_list()

In [126]:
tl = df2[(df2['Effect'] == 'synonymous') & (df2['QTL'] == df3[df3['ID'] == eyed]['QTL'].to_list()[0])].drop_duplicates('ID')['ID'].to_list()
tl = [s for s in tl if s in list(df3['ID'])]

In [123]:
df3.index = df3['ID'].values

In [138]:
eyed

7535187

In [140]:
ts = df3[df3['ID'] == eyed]

In [159]:
tdf.iloc[1:5,:]

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,IRIS_313-10340,...,IRIS_313-9422,IRIS_313-9445,IRIS_313-9594,IRIS_313-9606,IRIS_313-9617,IRIS_313-9626,IRIS_313-9661,IRIS_313-9682,IRIS_313-9696,QTL
1,chr06,20689079,7534659,G,A,.,PASS,.,GT,0,...,0,0,2,2,2,0,0,0,2,qCDP7
2,chr06,20689085,7534660,G,C,.,PASS,.,GT,2,...,2,2,0,0,0,2,2,2,0,qCDP7
3,chr06,20689191,7534663,T,C,.,PASS,.,GT,2,...,2,2,0,0,0,2,2,2,0,qCDP7
4,chr06,20689267,7534668,C,T,.,PASS,.,GT,0,...,0,0,2,2,2,0,0,0,2,qCDP7


In [46]:
df3 = df.iloc[:, 9:]

In [47]:
df3.index = df3

,IRIS_313-10340,IRIS_313-10341,IRIS_313-10537,IRIS_313-10539,IRIS_313-10587,IRIS_313-10592,IRIS_313-10593,IRIS_313-10595,IRIS_313-10598,IRIS_313-10600,...,IRIS_313-9397,IRIS_313-9422,IRIS_313-9445,IRIS_313-9594,IRIS_313-9606,IRIS_313-9617,IRIS_313-9626,IRIS_313-9661,IRIS_313-9682,IRIS_313-9696
0,2,2,0,2,2,2,2,2,2,2,...,2,2,0,2,2,2,2,2,0,2
1,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2,0,2,2,2,2,2,2,2,...,2,2,0,2,2,2,2,2,0,2
3,2,2,0,2,2,2,2,2,2,2,...,2,2,0,2,2,2,2,2,0,2
4,2,2,0,2,2,2,2,2,2,2,...,2,2,0,2,2,2,2,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23701,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23702,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,1,2,2,2
23703,2,2,0,0,0,0,0,0,2,0,...,2,2,0,0,0,2,0,2,2,0
23704,0,0,0,2,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0


In [84]:
for eyed in df2[df2['Effect'] == 'nonsense'].drop_duplicates('MID')['MID'].to_list():
    ts = df3[df3['ID'] == eyed]
    tl = df2[(df2['Effect'] == 'synonymous') & 
             (df2['QTL'] == df3[df3['ID'] == eyed]['QTL'].to_list()[0])].drop_duplicates('MID')['MID'].to_list()
    tl = [s for s in tl if s in list(df3['ID'])]
    tdf = df3.loc[tl, :]
    tdf = tdf.append(ts)
    tdf = tdf.sort_values('POS').reset_index(drop = True)
    #tdf.index = tdf['ID'].values
    tdf = tdf[(tdf.iloc[:, 9:-1].sum(axis = 1) > 17) & (tdf.iloc[:, 9:-1].sum(axis = 1) < (352 - 17))]
    tdf = tdf.iloc[(tdf[tdf['MID'] == eyed].index[0] - 10):(tdf[tdf['ID'] == eyed].index + 10)[0], :]
    tdf.iloc[:,9:-1].transpose().corr()
    break

KeyError: 'MID'

In [12]:
pd.read_csv('../Figures and tables/340 substitution effects annotated.csv')['Expression cluster']

array([ 1,  2,  3,  4,  5,  6,  9,  7, 10,  8], dtype=int64)

In [10]:
pd.merge(pd.read_excel('../Figures and tables/340 substitution effects.xlsx'),
    pd.read_csv('../Figures and tables/SNP effects annotated.csv')[['Transcript', 
        'Expression cluster', 'Functionally annotated', 'Outlier in expression', 'annotation']]).drop_duplicates().to_csv('../Figures and tables/340 substitution effects annotated.csv', index = False)